## Playcall API Requests

In [1]:
# import dependencies
import pandas as pd
import numpy as np
import requests
import json
from time import sleep
import matplotlib.pyplot as plt

In [2]:
# read in team data
cfbTeams = pd.read_csv("Data/fbsSchoolData.csv")

# set stub api url
urlstub = "https://api.collegefootballdata.com"

In [3]:
# create list to iterate through for years 2004-2019
years = np.arange(2004, 2020, 1)

In [4]:
fbsPlayCalls = pd.DataFrame({})
yearPlayCalls = pd.DataFrame({})

# run the same api call for all years since 2004
for y in years:
    yearPlayCalls = pd.DataFrame()
    schoolPlayCalls = pd.DataFrame()
    query_url = f"{urlstub}/stats/season/advanced?year={y}"
    resp = requests.get(query_url)
    respjs = resp.json()
    
    # iterate through the response to get the data for each school in each year
    for i in range(len(respjs)):
        
        try:            
            # create dictionary with column data specified
            playCalls = {"season": respjs[i]["season"],
                         "team": respjs[i]["team"],
                         "conference": respjs[i]["conference"],
                         "totalPlays": respjs[i]["offense"]["plays"],
                         "passingPlays": respjs[i]["offense"]["passingPlays"]["rate"] * respjs[i]["offense"]["plays"],
                         "rushingPlays": respjs[i]["offense"]["rushingPlays"]["rate"] * respjs[i]["offense"]["plays"]}

            # store per school and append to overall year frame
            schoolPlayCalls = pd.DataFrame(playCalls, index = [i])
            yearPlayCalls = yearPlayCalls.append(schoolPlayCalls, ignore_index=True, sort=False)

            #  append to overall dataframe
            fbsPlayCalls = fbsPlayCalls.append(yearPlayCalls, ignore_index=True, sort=False)
        
        except:
            print(f"Error in {y}: {resp}")
    
# Remove duplicate rows from the dataset, reset the index, and drop the previous index column
fbsPlayCalls = fbsPlayCalls.drop_duplicates()
fbsPlayCalls = fbsPlayCalls.reset_index()
del fbsPlayCalls["index"]

In [5]:
# Preview the dataset
fbsPlayCalls.head(10)

,season,team,conference,totalPlays,passingPlays,rushingPlays
0,2004,Air Force,Mountain West,147,33.0,110.0
1,2004,Akron,Mid-American,131,70.0,55.0
2,2004,Alabama,SEC,677,224.0,434.0
3,2004,Arizona,Pac-12,681,314.0,354.0
4,2004,Arizona State,Pac-12,823,429.0,370.0
5,2004,Arkansas,SEC,699,279.0,406.0
6,2004,Arkansas State,Sun Belt,268,140.0,120.0
7,2004,Army,FBS Independents,67,30.0,34.0
8,2004,Auburn,SEC,753,279.0,452.0
9,2004,Ball State,Mid-American,175,81.0,91.0


In [6]:
# Write data to csv
fbsPlayCalls.to_csv('Data/fbsPlayCallData.csv', index = False)